# 🎤 Clone Voice với RVC (Retrieval-based Voice Conversion)
Notebook để train và sử dụng RVC clone giọng nói trên Google Colab

**Yêu cầu:** GPU T4/A100 (chọn Runtime > Change runtime type > GPU)

In [ ]:
#@title 1️⃣ Kiểm tra GPU
!nvidia-smi

In [ ]:
#@title 2️⃣ Clone RVC và cài đặt dependencies
%cd /content

# Clone RVC WebUI
!git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI.git
%cd Retrieval-based-Voice-Conversion-WebUI

# Cài dependencies
!pip install -q -r requirements.txt
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q faiss-gpu
!pip install -q gradio==3.50.2

In [ ]:
#@title 3️⃣ Download pretrained models
%cd /content/Retrieval-based-Voice-Conversion-WebUI

# Download Hubert model
!mkdir -p assets/hubert
!wget -q -O assets/hubert/hubert_base.pt https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt

# Download RMVPE model (pitch extraction)
!mkdir -p assets/rmvpe
!wget -q -O assets/rmvpe/rmvpe.pt https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/rmvpe.pt

# Download pretrained models
!mkdir -p assets/pretrained_v2
!wget -q -O assets/pretrained_v2/f0G48k.pth https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0G48k.pth
!wget -q -O assets/pretrained_v2/f0D48k.pth https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0D48k.pth

print("✅ Đã download xong pretrained models!")

In [ ]:
#@title 4️⃣ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Tạo thư mục lưu trữ
!mkdir -p /content/drive/MyDrive/rvc_training/audio
!mkdir -p /content/drive/MyDrive/rvc_training/models
print("✅ Đã mount Drive và tạo thư mục!")

In [ ]:
#@title 5️⃣ Cấu hình Training

# === CẤU HÌNH CHÍNH ===
MODEL_NAME = "my_voice"  #@param {type:"string"}

# Thư mục chứa file audio training (upload vào Drive)
AUDIO_DIR = "/content/drive/MyDrive/rvc_training/audio"  #@param {type:"string"}
OUTPUT_DIR = "/content/drive/MyDrive/rvc_training/models"  #@param {type:"string"}

# Training parameters
SAMPLE_RATE = 48000  #@param [40000, 48000]
BATCH_SIZE = 8  #@param {type:"integer"}
TOTAL_EPOCHS = 100  #@param {type:"integer"}
SAVE_EVERY_EPOCH = 10  #@param {type:"integer"}

# F0 method
F0_METHOD = "rmvpe"  #@param ["rmvpe", "crepe", "harvest"]

print(f"✅ Cấu hình xong! Model: {MODEL_NAME}")

In [ ]:
#@title 6️⃣ Tiền xử lý Audio
%cd /content/Retrieval-based-Voice-Conversion-WebUI

import os
import shutil

# Copy audio files vào thư mục training
train_dir = f"logs/{MODEL_NAME}"
os.makedirs(train_dir, exist_ok=True)

# Kiểm tra audio files
audio_files = [f for f in os.listdir(AUDIO_DIR) if f.endswith(('.wav', '.mp3', '.flac', '.ogg'))]
print(f"Tìm thấy {len(audio_files)} file audio:")
for f in audio_files:
    print(f"  - {f}")

if len(audio_files) == 0:
    print("\n❌ Không tìm thấy file audio!")
    print(f"Hãy upload file audio (.wav, .mp3) vào: {AUDIO_DIR}")
else:
    # Process audio
    !python infer/modules/train/preprocess.py \
        "{AUDIO_DIR}" \
        {SAMPLE_RATE} \
        2 \
        "logs/{MODEL_NAME}" \
        False \
        3.0
    
    print("\n✅ Đã tiền xử lý audio xong!")

In [ ]:
#@title 7️⃣ Extract Features (Hubert + F0)
%cd /content/Retrieval-based-Voice-Conversion-WebUI

# Extract Hubert features
!python infer/modules/train/extract/extract_feature_print.py \
    cuda:0 \
    1 \
    0 \
    0 \
    "logs/{MODEL_NAME}" \
    "v2"

print("\n✅ Đã extract Hubert features!")

# Extract F0 (pitch)
!python infer/modules/train/extract_f0_print.py \
    "logs/{MODEL_NAME}" \
    2 \
    "{F0_METHOD}"

print("✅ Đã extract F0 features!")

In [ ]:
#@title 8️⃣ 🚀 Bắt đầu Training!
%cd /content/Retrieval-based-Voice-Conversion-WebUI

!python infer/modules/train/train.py \
    -e "{MODEL_NAME}" \
    -sr {SAMPLE_RATE} \
    -f0 1 \
    -bs {BATCH_SIZE} \
    -te {TOTAL_EPOCHS} \
    -se {SAVE_EVERY_EPOCH} \
    -pg "assets/pretrained_v2/f0G48k.pth" \
    -pd "assets/pretrained_v2/f0D48k.pth" \
    -l 0 \
    -c 0 \
    -sw 0 \
    -v "v2"

In [ ]:
#@title 9️⃣ Tạo Index file
%cd /content/Retrieval-based-Voice-Conversion-WebUI

!python infer/modules/train/extract_index.py \
    "logs/{MODEL_NAME}" \
    "v2"

print("✅ Đã tạo index file!")

In [ ]:
#@title 🔟 Copy model ra Drive
import shutil
import glob

# Tìm model file mới nhất
model_files = glob.glob(f"assets/weights/{MODEL_NAME}*.pth")
index_files = glob.glob(f"logs/{MODEL_NAME}/*.index")

for f in model_files:
    shutil.copy(f, OUTPUT_DIR)
    print(f"✅ Đã copy: {f}")

for f in index_files:
    shutil.copy(f, OUTPUT_DIR)
    print(f"✅ Đã copy: {f}")

print(f"\n📁 Models đã được lưu tại: {OUTPUT_DIR}")

In [ ]:
#@title 1️⃣1️⃣ Test Voice Conversion
%cd /content/Retrieval-based-Voice-Conversion-WebUI

import sys
sys.path.append('/content/Retrieval-based-Voice-Conversion-WebUI')

from infer.modules.vc.modules import VC
from configs.config import Config
import soundfile as sf
from IPython.display import Audio, display

# Load model
config = Config()
vc = VC(config)

# Tìm model path
import glob
model_path = glob.glob(f"assets/weights/{MODEL_NAME}*.pth")[0]
index_path = glob.glob(f"logs/{MODEL_NAME}/*.index")[0]

vc.get_vc(model_path)

# File audio test (thay đổi path này)
TEST_AUDIO = "/content/test_input.wav"  #@param {type:"string"}
OUTPUT_AUDIO = "/content/test_output.wav"

# Convert voice
result = vc.vc_single(
    sid=0,
    input_audio_path=TEST_AUDIO,
    f0_up_key=0,  # Pitch shift (semitones)
    f0_method=F0_METHOD,
    index_file=index_path,
    index_rate=0.75,
    protect=0.33
)

if result[0] == "Success":
    sf.write(OUTPUT_AUDIO, result[1][1], result[1][0])
    print("✅ Đã convert voice!")
    display(Audio(OUTPUT_AUDIO))
else:
    print(f"❌ Lỗi: {result[0]}")

In [ ]:
#@title 1️⃣2️⃣ Chạy WebUI (Optional)
%cd /content/Retrieval-based-Voice-Conversion-WebUI

# Chạy web interface
!python infer-web.py --pycmd python --port 7865 --share

## 📝 Hướng dẫn sử dụng:

### Chuẩn bị Audio Training:
1. **Số lượng:** 10-30 phút audio (tốt nhất 20+ phút)
2. **Chất lượng:**
   - Audio sạch, không có nhạc nền
   - Không có tiếng ồn, echo
   - Giọng nói rõ ràng
3. **Format:** `.wav` hoặc `.mp3` (khuyến nghị `.wav` 48kHz)
4. **Upload:** Đặt vào `/content/drive/MyDrive/rvc_training/audio`

### Thời gian train:
- 100 epochs ~ 1-2 giờ (T4 GPU)
- Thường 200-300 epochs cho kết quả tốt

### Sử dụng model:
1. Download file `.pth` và `.index` từ Drive
2. Dùng với RVC WebUI hoặc inference script
3. Có thể tích hợp với TTS để tạo giọng đọc

### Tips:
- **index_rate:** 0.5-0.8 (cao = giống voice gốc hơn)
- **f0_up_key:** Điều chỉnh pitch (0 = giữ nguyên, +12 = tăng 1 octave)
- **protect:** 0.3-0.5 (bảo vệ consonants)